In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
spark = SparkSession.builder \
  .appName('clean_products') \
  .config('spark.jars', 'gs://spark-lib/bigquery/spark-bigquery-latest_2.12.jar') \
  .getOrCreate()

In [2]:
spark.conf.set("spark.sql.repl.eagerEval.enable",True)

In [3]:
table_amazon_daily = "amazon_daily_updates.compras"
amazon_daily = spark.read \
  .format("bigquery") \
  .option("table", table_amazon_daily) \
  .load()
amazon_daily.printSchema()

root
 |-- id: string (nullable = false)
 |-- client_id: string (nullable = false)
 |-- product_id: string (nullable = false)
 |-- cantidad: long (nullable = false)
 |-- precio: double (nullable = false)
 |-- envio_id: string (nullable = true)
 |-- isprime: string (nullable = true)
 |-- fecha_compra: date (nullable = false)
 |-- metodo_pago: string (nullable = true)



In [4]:
from pyspark.sql.functions import current_date, date_sub

In [5]:
amazon_daily_2=amazon_daily.filter(amazon_daily['fecha_compra']==date_sub(current_date(),1))

In [6]:
amazon_daily_3=amazon_daily_2.withColumn("isprime",amazon_daily_2.isprime.cast('boolean')) \
    .withColumnRenamed("id","compra_id")\
    .withColumn("cantidad",amazon_daily_2.cantidad.cast("int"))\
    .withColumn("fecha_compra",amazon_daily_2.fecha_compra.cast("TIMESTAMP"))

In [4]:
amazon_daily_3=amazon_daily.withColumn("isprime",amazon_daily.isprime.cast('boolean')) \
    .withColumnRenamed("id","compra_id")\
    .withColumn("cantidad",amazon_daily.cantidad.cast("int"))\
    .withColumn("fecha_compra",amazon_daily.fecha_compra.cast("TIMESTAMP"))

In [5]:
amazon_daily_3.write \
  .format("bigquery") \
  .option("table","becade_rgarciaf.stg_compras") \
  .option("temporaryGcsBucket", "amazon_bucket_ramiro") \
  .mode('append') \
  .save()